In [1]:
# Create a model for text classification using a neural network to assign class probabilities
import pandas as pd
import numpy as np
import re
from helpers import *
# from tensorflow.contrib import learn
#import torch
#from torch.autograd import Variable
#import torch.nn as nn
#import torch.nn.functional as F
#from torch.optim.lr_scheduler import ReduceLROnPlateau
# from gensim.parsing.preprocessing import remove_stopwords
# import Pattern


##### RUN FOLLOWING TO GENERATE THE CSV FILE WITH TWEET CONTENT AND HASHTAGS
##### OTHERWISE LOAD THE SAVED DATAFRAME

# data = pd.read_csv('../data/rus_troll_tweet_text.csv')
# english_tweets = data[data.tweet_language == 'en']
# hashtag_info = pd.read_csv('../data/rus_troll_tweet_stats.csv')[['tweetid', 'hashtags']]
# merged = pd.merge(english_tweets, hashtag_info, on='tweetid')[['tweet_text', 'hashtags']]

# # Remove tweets that have no hashtag
# merged.replace('[]',np.nan, inplace=True)
# merged.dropna(how='any', inplace=True)

# # Remove hashtags and other tags and grammar stuff
# data_clean=merged.replace('#\w+', '', regex=True).replace('@\w+', '', regex=True).replace('RT ', '', regex=True)
# data_clean.tweet_text = data_clean.tweet_text.apply(lambda x: re.sub(r'http\S+', '', x)).apply(lambda x: re.sub(r"'|\"|`|:|\?|~|,|\.", '', x))
# data_clean.hashtags = data_clean.hashtags.apply(lambda x: x[1:-1]) # make list in hashtag column
# duplicated = pd.DataFrame(data_clean.hashtags.str.split(', ').tolist(), index=data_clean.tweet_text).stack()
# duplicated = duplicated.reset_index()[[0, 'tweet_text']] # var1 variable is currently labeled 0
# duplicated.columns = ['hashtags', 'tweet_text'] # renaming var1

# # Show top hashtags
# with pd.option_context('display.max_rows', None, 'display.max_columns', None):
#     print(duplicated.hashtags.value_counts()[0:300])
    
# # Remove stop words
# duplicated.tweet_text = duplicated.tweet_text.apply(lambda x: remove_stopwords(x))


# Filter dataframe for hashtags identified to be in a topic

# Create list of relevant hashtags
hashtag_list = ['MAGA', 'world', 'breaking', 'Breaking', 'TopNews', 'Breaking', 'money' 'WakeUpAmerica', 'MakeAmericaGreatAgain', 
                'ARRESTObama', 'TrumpTrain', 'pjnet', 'PJNET', 'Trump2016',
                'maga', 'TrumpForPresident', 'TrumpBecause', 'ObamaLameDuck',
                'ICelebrateTrumpWith', 'BlackLivesMatter', 'BlackTwitter', 'blacklivesmatter', 
                'BlackHistoryMonth', 'BlackSkinIsNotACrime',  'blacktolive', 'BlackToLive', 'blacktwitter',
                'tech', 'science', 'entertainment', 'celebs', 'MyEmmyNominationWouldBe', 'soundcloud', 'iTunes',
                'rap', 'RAP', 'showbiz', 'Music', 'life', 'health', 'sports', 'hockey', 'baseball', 'NowPlaying',
                'Sports', 'MyOlympicSportWouldBe', 'ISIS', 'IslamKills',
                'StopIslam', 'Hillary', 'ThingsMoreTrustedThanHillary', 'NeverHillary', 'CrookedHillary', 'LockHerUp',
                'nukraine', 'Obama', 'ObamaWishlist',
                'world', 'Syria', 'Turkey', 'Brussels', 'Russia', 'Iraq', 'business', 'news', 'local', 
                'Fukushima2015', 'FukushimaAgain', 'FUKUSHIMA2015', 'fukushima2015', 'IHatePokemonGoBecause',
                'crime', 'PoliceBrutality', 'USFA', 'ArmyWomenProblems', '2A', 'NRA', 'BLM', 'tcot', 'TCOT', 'GOP',
               'GOPDebate']

# # Filter dataframe
# filtered = duplicated[duplicated.hashtags.isin(hashtag_list)]


# # Lemmatize the words and also keeping their context (verb, noun etc)

# from gensim.utils import lemmatize
# filtered.tweet_text = filtered.tweet_text.apply(lambda x: lemmatize(x, min_length=3))


# # change hashtag names to the topic names
# pd.options.mode.chained_assignment = None  # default='warn' 

# # General News
# filtered.hashtags.loc[filtered[(filtered.hashtags == 'news') |
#                                (filtered.hashtags == 'local')].index.values] = 'General News'

# # Fukushima
# filtered.hashtags.loc[filtered[(filtered.hashtags == 'Fukushima2015') |
#                                (filtered.hashtags == 'FukushimaAgain') |
#                               (filtered.hashtags == 'FUKUSHIMA2015') | 
#                               (filtered.hashtags == 'fukushima2015')].index.values] = 'Fukushima'

# # Crime
# filtered.hashtags.loc[filtered[(filtered.hashtags == 'crime') |
#                                (filtered.hashtags == 'PoliceBrutality')].index.values] = 'Crime'

# # Gun Related
# filtered.hashtags.loc[filtered[(filtered.hashtags == 'USFA') |
#                                (filtered.hashtags == 'ArmyWomenProblems')].index.values] = 'Gun Related'

# # Anti-Islam
# filtered.hashtags.loc[filtered[(filtered.hashtags == 'ISIS') |
#                                (filtered.hashtags == 'IslamKills') |
#                               (filtered.hashtags == 'StopIslam')].index.values] = 'Anti-Islam'

# # Anti-Hillary
# filtered.hashtags.loc[filtered[(filtered.hashtags == 'Hillary') |
#                                (filtered.hashtags == 'ThingsMoreTrustedThanHillary') |
#                               (filtered.hashtags == 'NeverHillary') |
#                               (filtered.hashtags == 'CrookedHillary')].index.values] = 'Anti-Hillary'


# # Foreign Countries
# filtered.hashtags.loc[filtered[(filtered.hashtags == 'world') |
#                                (filtered.hashtags == 'Syria') |
#                               (filtered.hashtags == 'Turkey') |
#                               (filtered.hashtags == 'Brussels') |
#                               (filtered.hashtags == 'Russia') |
#                               (filtered.hashtags == 'Iraq')].index.values] = 'Foreign Countries'

# # Entertainment
# filtered.hashtags.loc[filtered[(filtered.hashtags == 'entertainment') |
#                                (filtered.hashtags == 'celebs') |
#                               (filtered.hashtags == 'MyEmmyNominationWouldBe') |
#                               (filtered.hashtags == 'soundcloud') |
#                               (filtered.hashtags == 'iTunes') |
#                               (filtered.hashtags == 'rap')].index.values] = 'Entertainment'


# # Black Race Support
# filtered.hashtags.loc[filtered[(filtered.hashtags == 'BlackLivesMatter') |
#                                (filtered.hashtags == 'BlackTwitter') |
#                               (filtered.hashtags == 'blacklivesmatter') |
#                               (filtered.hashtags == 'BlackHistoryMonth') |
#                               (filtered.hashtags == 'BlackSkinIsNotACrime') |
#                               (filtered.hashtags == 'blacktolive') |
#                               (filtered.hashtags == 'BlackToLive') |
#                               (filtered.hashtags == 'blacktwitter')].index.values] = 'Black Support'

# # Trump Support
# filtered.hashtags.loc[filtered[(filtered.hashtags == 'MAGA') |
#                                (filtered.hashtags == 'WakeUpAmerica') |
#                               (filtered.hashtags == 'MakeAmericaGreatAgain') |
#                               (filtered.hashtags == 'ARRESTObama') |
#                               (filtered.hashtags == 'TrumpTrain') |
#                               (filtered.hashtags == 'pjnet') |
#                               (filtered.hashtags == 'PJNET') |
#                               (filtered.hashtags == 'Trump2016') |
#                               (filtered.hashtags == 'maga') |
#                               (filtered.hashtags == 'TrumpForPresident') |
#                               (filtered.hashtags == 'TrumpBecause') |
#                               (filtered.hashtags == 'ObamaLameDuck') |
#                               (filtered.hashtags == 'ICelebrateTrumpWith')].index.values] = 'Trump Support'


# # Technology
# filtered.hashtags.loc[filtered[(filtered.hashtags == 'tech') |
#                                (filtered.hashtags == 'science')].index.values] = 'Technology'

# # Health
# filtered.hashtags.loc[filtered[filtered.hashtags == 'health'].index.values] = 'Health'

# # Sports
# filtered.hashtags.loc[filtered[(filtered.hashtags == 'sports') |
#                                (filtered.hashtags == 'hockey') |
#                               (filtered.hashtags == 'Sports') |
#                               (filtered.hashtags == 'MyOlympicSportWouldBe')].index.values] = 'Sports'


# # Business
# filtered.hashtags.loc[filtered[filtered.hashtags == 'business'].index.values] = 'Business'

# # Video GAmes
# filtered.hashtags.loc[filtered[filtered.hashtags == 'IHatePokemonGoBecause'].index.values] = 'Video Games'

In [2]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Twokenize -- a tokenizer designed for Twitter text in English and some other European languages.
This tokenizer code has gone through a long history:
(1) Brendan O'Connor wrote original version in Python, http://github.com/brendano/tweetmotif
       TweetMotif: Exploratory Search and Topic Summarization for Twitter.
       Brendan O'Connor, Michel Krieger, and David Ahn.
       ICWSM-2010 (demo track), http://brenocon.com/oconnor_krieger_ahn.icwsm2010.tweetmotif.pdf
(2a) Kevin Gimpel and Daniel Mills modified it for POS tagging for the CMU ARK Twitter POS Tagger
(2b) Jason Baldridge and David Snyder ported it to Scala
(3) Brendan bugfixed the Scala port and merged with POS-specific changes
    for the CMU ARK Twitter POS Tagger
(4) Tobi Owoputi ported it back to Java and added many improvements (2012-06)
Current home is http://github.com/brendano/ark-tweet-nlp and http://www.ark.cs.cmu.edu/TweetNLP
There have been at least 2 other Java ports, but they are not in the lineage for the code here.
Ported to Python by Myle Ott <myleott@gmail.com>.
"""
from __future__ import unicode_literals

import operator
import re
import sys

try:
    from html.parser import HTMLParser
except ImportError:
    from HTMLParser import HTMLParser

try:
    import html
except ImportError:
    pass

def regex_or(*items):
    return '(?:' + '|'.join(items) + ')'

Contractions = re.compile(u"(?i)(\w+)(n['’′]t|['’′]ve|['’′]ll|['’′]d|['’′]re|['’′]s|['’′]m)$", re.UNICODE)
Whitespace = re.compile(u"[\s\u0020\u00a0\u1680\u180e\u202f\u205f\u3000\u2000-\u200a]+", re.UNICODE)

punctChars = r"['\"“”‘’.?!…,:;]"
#punctSeq   = punctChars+"+"	#'anthem'. => ' anthem '.
punctSeq   = r"['\"“”‘’]+|[.?!,…]+|[:;]+"	#'anthem'. => ' anthem ' .
entity     = r"&(?:amp|lt|gt|quot);"
#  URLs


# BTO 2012-06: everyone thinks the daringfireball regex should be better, but they're wrong.
# If you actually empirically test it the results are bad.
# Please see https://github.com/brendano/ark-tweet-nlp/pull/9

urlStart1  = r"(?:https?://|\bwww\.)"
commonTLDs = r"(?:com|org|edu|gov|net|mil|aero|asia|biz|cat|coop|info|int|jobs|mobi|museum|name|pro|tel|travel|xxx)"
ccTLDs	 = r"(?:ac|ad|ae|af|ag|ai|al|am|an|ao|aq|ar|as|at|au|aw|ax|az|ba|bb|bd|be|bf|bg|bh|bi|bj|bm|bn|bo|br|bs|bt|" + \
r"bv|bw|by|bz|ca|cc|cd|cf|cg|ch|ci|ck|cl|cm|cn|co|cr|cs|cu|cv|cx|cy|cz|dd|de|dj|dk|dm|do|dz|ec|ee|eg|eh|" + \
r"er|es|et|eu|fi|fj|fk|fm|fo|fr|ga|gb|gd|ge|gf|gg|gh|gi|gl|gm|gn|gp|gq|gr|gs|gt|gu|gw|gy|hk|hm|hn|hr|ht|" + \
r"hu|id|ie|il|im|in|io|iq|ir|is|it|je|jm|jo|jp|ke|kg|kh|ki|km|kn|kp|kr|kw|ky|kz|la|lb|lc|li|lk|lr|ls|lt|" + \
r"lu|lv|ly|ma|mc|md|me|mg|mh|mk|ml|mm|mn|mo|mp|mq|mr|ms|mt|mu|mv|mw|mx|my|mz|na|nc|ne|nf|ng|ni|nl|no|np|" + \
r"nr|nu|nz|om|pa|pe|pf|pg|ph|pk|pl|pm|pn|pr|ps|pt|pw|py|qa|re|ro|rs|ru|rw|sa|sb|sc|sd|se|sg|sh|si|sj|sk|" + \
r"sl|sm|sn|so|sr|ss|st|su|sv|sy|sz|tc|td|tf|tg|th|tj|tk|tl|tm|tn|to|tp|tr|tt|tv|tw|tz|ua|ug|uk|us|uy|uz|" + \
r"va|vc|ve|vg|vi|vn|vu|wf|ws|ye|yt|za|zm|zw)"	#TODO: remove obscure country domains?
urlStart2  = r"\b(?:[A-Za-z\d-])+(?:\.[A-Za-z0-9]+){0,3}\." + regex_or(commonTLDs, ccTLDs) + r"(?:\."+ccTLDs+r")?(?=\W|$)"
urlBody    = r"(?:[^\.\s<>][^\s<>]*?)?"
urlExtraCrapBeforeEnd = regex_or(punctChars, entity) + "+?"
urlEnd     = r"(?:\.\.+|[<>]|\s|$)"
url        = regex_or(urlStart1, urlStart2) + urlBody + "(?=(?:"+urlExtraCrapBeforeEnd+")?"+urlEnd+")"


# Numeric
timeLike   = r"\d+(?::\d+){1,2}"
#numNum     = r"\d+\.\d+"
numberWithCommas = r"(?:(?<!\d)\d{1,3},)+?\d{3}" + r"(?=(?:[^,\d]|$))"
numComb	 = u"[\u0024\u058f\u060b\u09f2\u09f3\u09fb\u0af1\u0bf9\u0e3f\u17db\ua838\ufdfc\ufe69\uff04\uffe0\uffe1\uffe5\uffe6\u00a2-\u00a5\u20a0-\u20b9]?\\d+(?:\\.\\d+)+%?"

# Abbreviations
boundaryNotDot = regex_or("$", r"\s", r"[“\"?!,:;]", entity)
aa1  = r"(?:[A-Za-z]\.){2,}(?=" + boundaryNotDot + ")"
aa2  = r"[^A-Za-z](?:[A-Za-z]\.){1,}[A-Za-z](?=" + boundaryNotDot + ")"
standardAbbreviations = r"\b(?:[Mm]r|[Mm]rs|[Mm]s|[Dd]r|[Ss]r|[Jj]r|[Rr]ep|[Ss]en|[Ss]t)\."
arbitraryAbbrev = regex_or(aa1, aa2, standardAbbreviations)
separators  = "(?:--+|―|—|~|–|=)"
decorations = u"(?:[♫♪]+|[★☆]+|[♥❤♡]+|[\u2639-\u263b]+|[\ue001-\uebbb]+)"
thingsThatSplitWords = r"[^\s\.,?\"]"
embeddedApostrophe = thingsThatSplitWords+r"+['’′]" + thingsThatSplitWords + "*"

#  Emoticons
# myleott: in Python the (?iu) flags affect the whole expression
#normalEyes = "(?iu)[:=]" # 8 and x are eyes but cause problems
normalEyes = "[:=]" # 8 and x are eyes but cause problems
wink = "[;]"
noseArea = "(?:|-|[^a-zA-Z0-9 ])" # doesn't get 😢
happyMouths = r"[D\)\]\}]+"
sadMouths = r"[\(\[\{]+"
tongue = "[pPd3]+"
otherMouths = r"(?:[oO]+|[/\\]+|[vV]+|[Ss]+|[|]+)" # remove forward slash if http://'s aren't cleaned

# mouth repetition examples:
# @aliciakeys Put it in a love song :-))
# @hellocalyclops =))=))=)) Oh well

# myleott: try to be as case insensitive as possible, but still not perfect, e.g., o.O fails
#bfLeft = u"(♥|0|o|°|v|\\$|t|x|;|\u0ca0|@|ʘ|•|・|◕|\\^|¬|\\*)".encode('utf-8')
bfLeft = u"(♥|0|[oO]|°|[vV]|\\$|[tT]|[xX]|;|\u0ca0|@|ʘ|•|・|◕|\\^|¬|\\*)"
bfCenter = r"(?:[\.]|[_-]+)"
bfRight = r"\2"
s3 = r"(?:--['\"])"
s4 = r"(?:<|&lt;|>|&gt;)[\._-]+(?:<|&lt;|>|&gt;)"
s5 = "(?:[.][_]+[.])"
# myleott: in Python the (?i) flag affects the whole expression
#basicface = "(?:(?i)" +bfLeft+bfCenter+bfRight+ ")|" +s3+ "|" +s4+ "|" + s5
basicface = "(?:" +bfLeft+bfCenter+bfRight+ ")|" +s3+ "|" +s4+ "|" + s5

eeLeft = r"[＼\\ƪԄ\(（<>;ヽ\-=~\*]+"
eeRight= u"[\\-=\\);'\u0022<>ʃ）/／ノﾉ丿╯σっµ~\\*]+"
eeSymbol = r"[^A-Za-z0-9\s\(\)\*:=-]"
eastEmote = eeLeft + "(?:"+basicface+"|" +eeSymbol+")+" + eeRight

oOEmote = r"(?:[oO]" + bfCenter + r"[oO])"


emoticon = regex_or(
        # Standard version  :) :( :] :D :P
        "(?:>|&gt;)?" + regex_or(normalEyes, wink) + regex_or(noseArea,"[Oo]") + regex_or(tongue+r"(?=\W|$|RT|rt|Rt)", otherMouths+r"(?=\W|$|RT|rt|Rt)", sadMouths, happyMouths),

        # reversed version (: D:  use positive lookbehind to remove "(word):"
        # because eyes on the right side is more ambiguous with the standard usage of : ;
        regex_or("(?<=(?: ))", "(?<=(?:^))") + regex_or(sadMouths,happyMouths,otherMouths) + noseArea + regex_or(normalEyes, wink) + "(?:<|&lt;)?",

        #inspired by http://en.wikipedia.org/wiki/User:Scapler/emoticons#East_Asian_style
        eastEmote.replace("2", "1", 1), basicface,
        # iOS 'emoji' characters (some smileys, some symbols) [\ue001-\uebbb]
        # TODO should try a big precompiled lexicon from Wikipedia, Dan Ramage told me (BTO) he does this

        # myleott: o.O and O.o are two of the biggest sources of differences
        #          between this and the Java version. One little hack won't hurt...
        oOEmote
)

Hearts = "(?:<+/?3+)+" #the other hearts are in decorations

Arrows = regex_or(r"(?:<*[-―—=]*>+|<+[-―—=]*>*)", u"[\u2190-\u21ff]+")

# BTO 2011-06: restored Hashtag, AtMention protection (dropped in original scala port) because it fixes
# "hello (#hashtag)" ==> "hello (#hashtag )"  WRONG
# "hello (#hashtag)" ==> "hello ( #hashtag )"  RIGHT
# "hello (@person)" ==> "hello (@person )"  WRONG
# "hello (@person)" ==> "hello ( @person )"  RIGHT
# ... Some sort of weird interaction with edgepunct I guess, because edgepunct
# has poor content-symbol detection.

# This also gets #1 #40 which probably aren't hashtags .. but good as tokens.
# If you want good hashtag identification, use a different regex.
Hashtag = "#[a-zA-Z0-9_]+"  #optional: lookbehind for \b
#optional: lookbehind for \b, max length 15
AtMention = "[@＠][a-zA-Z0-9_]+"

# I was worried this would conflict with at-mentions
# but seems ok in sample of 5800: 7 changes all email fixes
# http://www.regular-expressions.info/email.html
Bound = r"(?:\W|^|$)"
Email = regex_or("(?<=(?:\W))", "(?<=(?:^))") + r"[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,4}(?=" +Bound+")"

# We will be tokenizing using these regexps as delimiters
# Additionally, these things are "protected", meaning they shouldn't be further split themselves.
Protected  = re.compile(
    regex_or(
        Hearts,
        url,
        Email,
        timeLike,
        #numNum,
        numberWithCommas,
        numComb,
        emoticon,
        Arrows,
        entity,
        punctSeq,
        arbitraryAbbrev,
        separators,
        decorations,
        embeddedApostrophe,
        Hashtag,
        AtMention), re.UNICODE)

# Edge punctuation
# Want: 'foo' => ' foo '
# While also:   don't => don't
# the first is considered "edge punctuation".
# the second is word-internal punctuation -- don't want to mess with it.
# BTO (2011-06): the edgepunct system seems to be the #1 source of problems these days.
# I remember it causing lots of trouble in the past as well.  Would be good to revisit or eliminate.

# Note the 'smart quotes' (http://en.wikipedia.org/wiki/Smart_quotes)
#edgePunctChars    = r"'\"“”‘’«»{}\(\)\[\]\*&" #add \\p{So}? (symbols)
edgePunctChars    = u"'\"“”‘’«»{}\\(\\)\\[\\]\\*&" #add \\p{So}? (symbols)
edgePunct    = "[" + edgePunctChars + "]"
notEdgePunct = "[a-zA-Z0-9]" # content characters
offEdge = r"(^|$|:|;|\s|\.|,)"  # colon here gets "(hello):" ==> "( hello ):"
EdgePunctLeft  = re.compile(offEdge + "("+edgePunct+"+)("+notEdgePunct+")", re.UNICODE)
EdgePunctRight = re.compile("("+notEdgePunct+")("+edgePunct+"+)" + offEdge, re.UNICODE)

def splitEdgePunct(input):
    input = EdgePunctLeft.sub(r"\1\2 \3", input)
    input = EdgePunctRight.sub(r"\1 \2\3", input)
    return input

# The main work of tokenizing a tweet.
def simpleTokenize(text):

    # Do the no-brainers first
    splitPunctText = splitEdgePunct(text)

    textLength = len(splitPunctText)

    # BTO: the logic here got quite convoluted via the Scala porting detour
    # It would be good to switch back to a nice simple procedural style like in the Python version
    # ... Scala is such a pain.  Never again.

    # Find the matches for subsequences that should be protected,
    # e.g. URLs, 1.0, U.N.K.L.E., 12:53
    bads = []
    badSpans = []
    for match in Protected.finditer(splitPunctText):
        # The spans of the "bads" should not be split.
        if (match.start() != match.end()): #unnecessary?
            bads.append( [splitPunctText[match.start():match.end()]] )
            badSpans.append( (match.start(), match.end()) )

    # Create a list of indices to create the "goods", which can be
    # split. We are taking "bad" spans like
    #     List((2,5), (8,10))
    # to create
    #     List(0, 2, 5, 8, 10, 12)
    # where, e.g., "12" here would be the textLength
    # has an even length and no indices are the same
    indices = [0]
    for (first, second) in badSpans:
        indices.append(first)
        indices.append(second)
    indices.append(textLength)

    # Group the indices and map them to their respective portion of the string
    splitGoods = []
    for i in range(0, len(indices), 2):
        goodstr = splitPunctText[indices[i]:indices[i+1]]
        splitstr = goodstr.strip().split(" ")
        splitGoods.append(splitstr)

    #  Reinterpolate the 'good' and 'bad' Lists, ensuring that
    #  additonal tokens from last good item get included
    zippedStr = []
    for i in range(len(bads)):
        zippedStr = addAllnonempty(zippedStr, splitGoods[i])
        zippedStr = addAllnonempty(zippedStr, bads[i])
    zippedStr = addAllnonempty(zippedStr, splitGoods[len(bads)])

    # BTO: our POS tagger wants "ur" and "you're" to both be one token.
    # Uncomment to get "you 're"
    #splitStr = []
    #for tok in zippedStr:
    #    splitStr.extend(splitToken(tok))
    #zippedStr = splitStr

    return zippedStr

def addAllnonempty(master, smaller):
    for s in smaller:
        strim = s.strip()
        if (len(strim) > 0):
            master.append(strim)
    return master

# "foo   bar " => "foo bar"
def squeezeWhitespace(input):
    return Whitespace.sub(" ", input).strip()

# Final pass tokenization based on special patterns
def splitToken(token):
    m = Contractions.search(token)
    if m:
        return [m.group(1), m.group(2)]
    return [token]

# Assume 'text' has no HTML escaping.
def tokenize(text):
    return simpleTokenize(squeezeWhitespace(text))


# Twitter text comes HTML-escaped, so unescape it.
# We also first unescape &amp;'s, in case the text has been buggily double-escaped.
def normalizeTextForTagger(text):
    assert sys.version_info[0] >= 3 and sys.version_info[1] > 3, 'Python version >3.3 required'
    text = text.replace("&amp;", "&")
    text = html.unescape(text)
    return text

# This is intended for raw tweet text -- we do some HTML entity unescaping before running the tagger.
#
# This function normalizes the input text BEFORE calling the tokenizer.
# So the tokens you get back may not exactly correspond to
# substrings of the original text.
def tokenizeRawTweetText(text):
    tokens = tokenize(normalizeTextForTagger(text))
    return tokens

In [ ]:
data = filtered[(filtered.hashtags == 'Sports') |
               (filtered.hashtags == 'Health') |
                (filtered.hashtags == 'Trump Support') |
                (filtered.hashtags == 'Black Support') |
                (filtered.hashtags == 'Entertainment') |
                (filtered.hashtags == 'Foreign Countries') |
                (filtered.hashtags == 'Anti-Trump') |
                (filtered.hashtags == 'Health') |
                (filtered.hashtags == 'Anti-Islam') |
                (filtered.hashtags == 'Patriot') |
                (filtered.hashtags == 'Crime') |
                (filtered.hashtags == 'Fukushima') |
                (filtered.hashtags == 'News')]

In [7]:
stopwords2 = open(access_folder() + 'morestopwords.txt', "r").read().split('\n')
stopwords2 = list(filter(None, stopwords2)) # fastest
stopwords2 = [word for word in stopwords2 if word[0]!='#' ]
stopwords3 = open(access_folder() + 'common-english-prep-conj.txt' , "r").read().split(',')
stopwords4 = open(access_folder() + 'common-english-words.txt' , "r").read().split(',')

stopwords = []
stopwords = stopwords + stopwords2 +  stopwords3 + stopwords4

In [4]:
# Load english dataframe
filtered = pd.read_csv('../generated/tweets_n_hashtags.csv', index_col = 0)

In [5]:
filtered.head()

,hashtags,tweet_text
33,news,"['john', 'carroll', 'university', 'gets', '$22..."
34,local,"['spring', 'cook', 'book', 'healthy', 'chocola..."
37,news,"['rocky', 'river', 'prepares', 'more', 'floodi..."
39,news,"['forecast', 'cooler', 'mid-week', 'cooler', '..."
40,TopNews,"['brothers', 'farook', 'one', 'decorated', 've..."


In [6]:
filtered.tweet_text = filtered.tweet_text.apply(lambda x: x[1:-1]).replace("b'", '', regex=True).replace("'", '', regex=True)
filtered.tweet_text = filtered.tweet_text.apply(lambda x: x.split(', '))

In [14]:
# News
filtered.hashtags.loc[filtered[(filtered.hashtags == 'news') |
                               (filtered.hashtags == 'local') |
                               (filtered.hashtags == 'isis') |
                               (filtered.hashtags == 'world') |
                               (filtered.hashtags == 'breaking') |
                               (filtered.hashtags == 'business') |
                               (filtered.hashtags == 'TopNews') |
                               (filtered.hashtags == 'Breaking') |
                               (filtered.hashtags == 'money')].index.values] = 'News'

# Fukushima
filtered.hashtags.loc[filtered[(filtered.hashtags == 'Fukushima2015') |
                               (filtered.hashtags == 'FukushimaAgain') |
                              (filtered.hashtags == 'FUKUSHIMA2015') | 
                              (filtered.hashtags == 'fukushima2015')].index.values] = 'Fukushima'

# Crime
filtered.hashtags.loc[filtered[(filtered.hashtags == 'crime') |
                               (filtered.hashtags == 'PoliceBrutality')].index.values] = 'Crime'

# Patriot
filtered.hashtags.loc[filtered[(filtered.hashtags == 'NRA') |
                               (filtered.hashtags == 'USFA') |
                               (filtered.hashtags == 'NRA') |
                               (filtered.hashtags == 'ArmyWomenProblems') |
                              (filtered.hashtags == 'pjnet') |
                              (filtered.hashtags == 'PJNET') |
                              (filtered.hashtags == '2A')].index.values] = 'Patriot'


# Anti-Islam
filtered.hashtags.loc[filtered[(filtered.hashtags == 'ISIS') |
                               (filtered.hashtags == 'IslamKills') |
                              (filtered.hashtags == 'StopIslam')].index.values] = 'Anti-Islam'

# Anti-Trump
filtered.hashtags.loc[filtered[(filtered.hashtags == 'Hillary') |
                               (filtered.hashtags == 'ThingsMoreTrustedThanHillary') |
                              (filtered.hashtags == 'NeverHillary') |
                               (filtered.hashtags == 'ARRESTObama') |
                               (filtered.hashtags == 'LockHerUp') |
                               (filtered.hashtags == 'ObamaWishlist') |
                               (filtered.hashtags == 'Obama') |
                              (filtered.hashtags == 'ObamaLameDuck') |
                              (filtered.hashtags == 'CrookedHillary')].index.values] = 'Anti-Trump'


# Foreign Countries
filtered.hashtags.loc[filtered[(filtered.hashtags == 'isis') |
                               (filtered.hashtags == 'Syria') |
                              (filtered.hashtags == 'Turkey') |
                               (filtered.hashtags == 'nukraine') |
                              (filtered.hashtags == 'Brussels') |
                              (filtered.hashtags == 'Russia') |
                              (filtered.hashtags == 'Iraq')].index.values] = 'Foreign Countries'

# Entertainment
filtered.hashtags.loc[filtered[(filtered.hashtags == 'entertainment') |
                               (filtered.hashtags == 'celebs') |
                               (filtered.hashtags == 'NowPlaying') |
                               (filtered.hashtags == 'Showbiz') |
                               (filtered.hashtags == 'showbiz') |
                               (filtered.hashtags == 'RAP') |
                               (filtered.hashtags == 'life') |
                               (filtered.hashtags == 'IHatePokemonGoBecause') |
                               (filtered.hashtags == 'Music') |
                              (filtered.hashtags == 'MyEmmyNominationWouldBe') |
                              (filtered.hashtags == 'soundcloud') |
                              (filtered.hashtags == 'iTunes') |
                              (filtered.hashtags == 'rap')].index.values] = 'Entertainment'


# Black Race Support
filtered.hashtags.loc[filtered[(filtered.hashtags == 'BlackLivesMatter') |
                               (filtered.hashtags == 'BlackTwitter') |
                               (filtered.hashtags == 'blacktolive') |
                              (filtered.hashtags == 'blacklivesmatter') |
                              (filtered.hashtags == 'BlackHistoryMonth') |
                              (filtered.hashtags == 'BlackSkinIsNotACrime') |
                              (filtered.hashtags == 'BLM') |
                              (filtered.hashtags == 'BlackToLive') |
                              (filtered.hashtags == 'blacktwitter')].index.values] = 'Black Support'

# Trump Support
filtered.hashtags.loc[filtered[(filtered.hashtags == 'MAGA') |
                               (filtered.hashtags == 'WakeUpAmerica') |
                              (filtered.hashtags == 'MakeAmericaGreatAgain') |
                              (filtered.hashtags == 'TrumpTrain') |
                              (filtered.hashtags == 'Trump2016') |
                              (filtered.hashtags == 'maga') |
                               (filtered.hashtags == 'tcot') |
                               (filtered.hashtags == 'TCOT') |
                               (filtered.hashtags == 'GOP') |
                               (filtered.hashtags == 'GOPDebate') |
                               (filtered.hashtags == 'ccot') |
                               (filtered.hashtags == 'AmericaFirst') |
                              (filtered.hashtags == 'TrumpForPresident') |
                              (filtered.hashtags == 'TrumpBecause') |
                              (filtered.hashtags == 'ICelebrateTrumpWith')].index.values] = 'Trump Support'


# Health
filtered.hashtags.loc[filtered[(filtered.hashtags == 'health') |
                              (filtered.hashtags == 'environment')].index.values] = 'Health'

# Sports
filtered.hashtags.loc[filtered[(filtered.hashtags == 'sports') |
                               (filtered.hashtags == 'hockey') |
                               (filtered.hashtags == 'Hockey') |
                               (filtered.hashtags == 'baseball') |
                              (filtered.hashtags == 'Sports') |
                              (filtered.hashtags == 'MyOlympicSportWouldBe')].index.values] = 'Sports'


In [22]:
filtered = filtered[filtered.hashtags != 'science']
filtered = filtered[filtered.hashtags != 'tech']

In [23]:
filtered.hashtags.unique()

array(['News', 'Sports', 'Crime', 'Entertainment', 'Anti-Trump',
       'Fukushima', 'Trump Support', 'Patriot', 'Health', 'Black Support',
       'Anti-Islam', 'Foreign Countries'], dtype=object)

In [37]:
filtered.tweet_text = filtered.tweet_text.apply(lambda x: ' '.join(x))

In [38]:
from gensim.utils import lemmatize
filtered.tweet_text = filtered.tweet_text.apply(lambda x: lemmatize(x, min_length=3))

In [ ]:
tweet_list = []
for tweet in filtered.tweet_text.values:
    sentence = []
    for word in tweet:
        sentence.append(word[0:-3].decode("utf-8"))
    tweet_list.append(sentence)

filtered.tweet_text = tweet_list

In [59]:
filtered.to_csv('tweets_n_hashtags.csv')

In [56]:
categories = ['News', 'Sports', 'Crime', 'Fukushima', 'Entertainment', 
              'Anti-Trump', 'Patriot', 'Trump Support',
              'Foreign Countries', 'Health',
              'Black Support', 'Anti-Islam']
from collections import Counter

vocab = Counter()

for text in filtered.tweet_text:
    for word in text: ## change this because the texts are already a list of words
        vocab[word[2:-1].lower()]+=1

total_words = len(vocab)

def get_word_2_index(vocab):
    word2index = {}
    for i,word in enumerate(vocab):
        word2index[word.lower()] = i

    return word2index

word2index = get_word_2_index(vocab)

In [57]:
total_words

35149

In [55]:
filtered

,hashtags,tweet_text
33,News,"[john, carroll, university, get, gift]"
34,News,"[spring, cook, book, healthy, chocolate, bar]"
37,News,"[rocky, river, prepare, more, flooding, city, ..."
39,News,"[forecast, cooler, mid, week, cooler, isolated..."
40,News,"[brother, farook, decorate, veteran, killer]"
42,News,"[arrest, connection, art, festival, theft, arr..."
44,News,"[director, craven, die, scream, filmmaker, die..."
45,Sports,"[secretariat, dominate, american, pharoah, bel..."
46,News,"[black, box, miss, german, airline, crash]"
48,Sports,"[dale, earnhardt, get, engage, girlfriend, amy..."


In [ ]:
def get_batch(df,i,batch_size):
    batches = []
    results = []
    texts = df.tweet_text[i*batch_size:i*batch_size+batch_size]
    categories = df.hashtags[i*batch_size:i*batch_size+batch_size]
    for text in texts:
        layer = np.zeros(total_words,dtype=float) # total words is not defined
        for word in text:
            layer[word2index[word[2:-1].lower()]] += 1

        batches.append(layer)

    for category in categories:
        index_y = -1
        if category == 'General News':
            index_y = 0
        elif category == 'Sports':
            index_y = 1
        elif category == 'Crime':
            index_y = 2
        elif category == 'Technology':
            index_y = 3
        elif category == 'Entertainment':
            index_y = 4
        elif category == 'Anti-Hillary':
            index_y = 5
        elif category == 'Fukushima':
            index_y = 6
        elif category == 'Foreign Countries':
            index_y = 7
        elif category == 'Trump Support':
            index_y = 8
        elif category == 'Gun Related':
            index_y = 9
        elif category == 'Video Games':
            index_y = 10
        elif category == 'Health':
            index_y = 11
        elif category == 'Business':
            index_y = 12
        elif category == 'Black Support':
            index_y = 13
        elif category == 'Anti-Islam':
            index_y = 14
        results.append(index_y)


    return np.array(batches),np.array(results)

In [ ]:
# Parameters
learning_rate = 0.01
num_epochs = 4
batch_size = 150
display_step = 1

# Network Parameters
hidden_size = 100      # 1st layer and 2nd layer number of features
input_size = total_words # Words in vocab
num_classes = 15        # Categories: graphics, sci.space and baseball

In [ ]:
loss = nn.CrossEntropyLoss()
inputs = Variable(torch.randn(2, 5), requires_grad=True)
target = Variable(torch.LongTensor(2).random_(5))
output = loss(inputs, target)
output.backward()

# Do Neural Network with Linear Layers

In [ ]:
class OurNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super().__init__()
        self.layer_1 = nn.Linear(input_size,hidden_size, bias=True)
        self.relu = nn.ReLU()
        self.layer_2 = nn.Linear(hidden_size, hidden_size, bias=True)
        self.output_layer = nn.Linear(hidden_size, num_classes, bias=True)

    def forward(self, x):
        out = self.layer_1(x)
        out = self.relu(out)
        out = self.layer_2(out)
        out = self.relu(out)
        out = self.output_layer(out)
        return out
    
net = OurNet(input_size, hidden_size, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate, momentum=0.9)
# scheduler = ReduceLROnPlateau(optimizer, 'min') # for validation set

In [ ]:
## Train the Model
for epoch in range(num_epochs):
    total_batch = int(len(filtered.tweet_text)/batch_size)
    # Loop over all batches
    for i in range(total_batch):
        batch_x,batch_y = get_batch(filtered,i,batch_size)
        articles = Variable(torch.FloatTensor(batch_x))
        labels = Variable(torch.LongTensor(batch_y))
        #print("articles",articles)
        #print(batch_x, labels)
        #print("size labels",labels.size())

        # Forward + Backward + Optimize
        optimizer.zero_grad()  # zero the gradient buffer
        outputs = net(articles)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print ('Epoch [%d/%d], Step [%d/%d], Loss: %.4f'
                   %(epoch+1, num_epochs, i+1, len(filtered.tweet_text)//batch_size, loss.data[0]))
            
    

In [ ]:
# lemmatize(['I need to get healthy who wants to go to the gym', 'Yesterday I saw a great football game'], min_length=3)

In [ ]:
test_set = filtered.loc[2150651].copy()

test_set

test_set.head()
batch_x_test,batch_y_test = get_batch(test_set,0,2)
articles = Variable(torch.FloatTensor(batch_x_test))
labels = torch.LongTensor(batch_y_test)
outputs = net(articles)
print(outputs)
_, predicted = torch.max(outputs.data, 1)
print(predicted)

# torch.save(net.state_dict(), 'supervised_linear')

# Try Convolutional Neural Net

In [ ]:
class ConvNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super().__init__()
        self.conv1 = nn.Conv1d(input_size, input_size, kernel_size=2) # dimensions need to be fixed
        self.conv2 = nn.Conv1d(hidden_size, hidden_size, kernel_size=2)
        self.mp = nn.MaxPool1d(2)
        self.fc = nn.Linear(100, 15)
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.mp(x)
        x = F.relu(x)
        x = F.relu(self.mp(self.conv2(x)))
        x = x.view(in_size, -1)
        x = self.fc(x)
        return F.log_softmax(x)
        
convnet = ConvNet(input_size, hidden_size, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)

In [ ]:
## Train the Model
for epoch in range(num_epochs):
    total_batch = int(len(filtered.tweet_text)/batch_size)
    # Loop over all batches
    for i in range(total_batch):
        batch_x,batch_y = get_batch(filtered,i,batch_size)
        articles = Variable(torch.FloatTensor(batch_x))
        labels = Variable(torch.LongTensor(batch_y))
        #print("articles",articles)
        #print(batch_x, labels)
        #print("size labels",labels.size())

        # Forward + Backward + Optimize
        optimizer.zero_grad()  # zero the gradient buffer
        outputs = convnet(articles)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print ('Epoch [%d/%d], Step [%d/%d], Loss: %.4f'
                   %(epoch+1, num_epochs, i+1, len(filtered.tweet_text)//batch_size, loss.data[0]))